## Function Calling / Tool Calling

In [1]:
from openai import AzureOpenAI

client = AzureOpenAI(api_version="2024-12-01-preview")

In [ ]:
response = client.chat.completions.create(model='telcogpt',
                                          messages=[{'role':'user',
                                                    'content':'Explain Quantum Computing in 2 lines'}],
                                        temperature=0.2)

print(response.to_json(indent=2))


In [9]:

import requests,json
def get_current_weather(city:str)->dict:
    """ can be used to get/fetch current weather information for a city name
    """
    api_key = "6a8b0ac166a37e2b7a38e64416b3c3fe"

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode()
    response = json.loads(response)
    output = {"City Name":city,"weather":response["weather"][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}

    return output


In [10]:
get_current_weather("mumbai")

{'City Name': 'mumbai',
 'weather': 'clear sky',
 'temperature': 303.26,
 'unit': 'kelvin'}

In [13]:
# name, description, parameters (also desc for parameters)

tools = [{"type":"function",
          "function":{
              "name":"get_current_weather_info",
              "description":"this function is used to get the current weather information for a given city",
              "parameters":{
                  "type":"object",
                  "properties":{"city":{"type":"string","description":"name of location/city, e.g. mumbai, new york"}}

              },
              "required":['city'],
          }}]

In [24]:
def generate_response(prompt):

    messages = [{"role":"system","content":"you are a helpful asistant"},
                {"role":"user","content":prompt}]
    first_response = client.chat.completions.create(model='telcogpt',
                                                    messages=messages,
                                                    temperature=0.1,tools=tools,
                                                    tool_choice='auto')
    
    if first_response.choices[0].message.tool_calls:
        print(first_response.choices[0].message.tool_calls)
        tool_calls = first_response.choices[0].message.tool_calls
        tool_map = {"get_current_weather_info":get_current_weather}
        messages.append(first_response.choices[0].message)
        for tool in tool_calls:
            tool_name = tool.function.name # fetching the function name (metadata reference)
            tool_args = json.loads(tool.function.arguments) # fetching funciton arguments
            function_to_ex = tool_map[tool_name] # getting the tool name
            tool_output = function_to_ex(**tool_args) # executing the function
            messages.append({"tool_call_id":tool.id,"role":"tool","name":tool_name,
                             "content":json.dumps(tool_output)})
        second_response = client.chat.completions.create(model="telcogpt",messages=messages,
                                                         temperature=0.1)
        return second_response.choices[0].message.content

    else:
        return first_response.choices[0].message.content

In [25]:
resp = generate_response("what is the current weather in mumbai")
print(resp)

[ChatCompletionMessageToolCall(id='call_Xnnb605u70kq2nPwMbAxCLpz', function=Function(arguments='{"city":"mumbai"}', name='get_current_weather_info'), type='function')]
The current weather in Mumbai is clear sky with a temperature of approximately 303.26 K (about 30.1 °C).


In [26]:
resp = generate_response("what is quantum computing")
print(resp)

Quantum computing is a type of computation that takes advantage of the principles of quantum mechanics to process information in fundamentally different ways than classical computers. Here are some key concepts:

1. **Quantum Bits (Qubits)**: Unlike classical bits, which can be either 0 or 1, qubits can exist in a state of superposition, meaning they can be both 0 and 1 simultaneously. This allows quantum computers to process a vast amount of information at once.

2. **Superposition**: This principle allows qubits to be in multiple states at the same time. When a qubit is in superposition, it can perform multiple calculations simultaneously, which can lead to faster problem-solving for certain tasks.

3. **Entanglement**: Qubits can be entangled, meaning the state of one qubit is directly related to the state of another, no matter the distance between them. This property can be used to link qubits in complex ways, enabling more powerful computations.

4. **Quantum Gates**: Quantum comp

In [ ]:
resp = generate_response("what is the current weather in mumbai and London")
print(resp)